此作業在Windows 10 下執行

In [ ]:
!pip freeze > requirements.txt

In [ ]:
import os

model_path = "./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"
print("檔案是否存在？", os.path.exists(model_path))

In [ ]:
import sys
print(sys.version)
print(sys.executable)

In [ ]:

import requests

def download_file(url, filename):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(filename, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"下載完成: {filename}")
    else:
        print(f"下載失敗: {url}")

# 下載 LLaMA 3.1 模型和 NTU 檔案
#download_file("https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf", "Meta-Llama-3.1-8B-Instruct-Q8_0.gguf")
#download_file("https://www.csie.ntu.edu.tw/~ulin/public.txt", "public.txt")
#download_file("https://www.csie.ntu.edu.tw/~ulin/private.txt", "private.txt")




In [ ]:
import torch
print(torch.__version__)  # 應該顯示 "+cu121"
print(torch.cuda.is_available())  # 應該回傳 True
print(torch.cuda.get_device_name(0))  # 應該顯示 "NVIDIA GeForce RTX 4060 ..."

In [ ]:
import torch
if not torch.cuda.is_available():
    raise Exception('You are not using the GPU runtime. Change it first or you will suffer from the super slow inference speed!')
else:
    print('You are good to go!')

In [ ]:
from llama_cpp import Llama

# Load the model onto GPU
llama3 = Llama(
    "./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf",
    verbose=False,
    n_gpu_layers=-1,
    n_ctx=16384,    # This argument is how many tokens the model can take. The longer the better, but it will consume more memory. 16384 is a proper value for a GPU with 16GB VRAM.
)

def generate_response(_model: Llama, _messages: str) -> str:
    '''
    This function will inference the model with given messages.
    '''
    _output = _model.create_chat_completion(
        _messages,
        stop=["<|eot_id|>", "<|end_of_text|>"],
        max_tokens=512,    # This argument is how many tokens the model can generate, you can change it and observe the differences.
        temperature=0,      # This argument is the randomness of the model. 0 means no randomness. You will get the same result with the same input every time. You can try to set it to different values.
        repeat_penalty=2.0,
    )["choices"][0]["message"]["content"]
    return _output

In [ ]:
from typing import List
from googlesearch import search as _search
from bs4 import BeautifulSoup
from charset_normalizer import detect
import asyncio
from requests_html import AsyncHTMLSession
import urllib3
urllib3.disable_warnings()
MAX_CHAR_LENGTH = 2000
async def worker(s:AsyncHTMLSession, url:str):
    try:
        header_response = await asyncio.wait_for(s.head(url, verify=False), timeout=10)
        if 'text/html' not in header_response.headers.get('Content-Type', ''):
            return None
        r = await asyncio.wait_for(s.get(url, verify=False), timeout=10)
        return r.text
    except:
        return None

async def get_htmls(urls):
    session = AsyncHTMLSession()
    tasks = (worker(session, url) for url in urls)
    return await asyncio.gather(*tasks)

async def search(keyword: str, n_results: int=3) -> List[str]:
    '''
    This function will search the keyword and return the text content in the first n_results web pages.

    Warning: You may suffer from HTTP 429 errors if you search too many times in a period of time. This is unavoidable and you should take your own risk if you want to try search more results at once.
    The rate limit is not explicitly announced by Google, hence there's not much we can do except for changing the IP or wait until Google unban you (we don't know how long the penalty will last either).
    '''
    keyword = keyword[:100]
    # First, search the keyword and get the results. Also, get 2 times more results in case some of them are invalid.
    results = list(_search(keyword, n_results * 1, region="tw",lang="zh", unique=True))
    # Then, get the HTML from the results. Also, the helper function will filter out the non-HTML urls.
    results = await get_htmls(results)
    # Filter out the None values.
    results = [x for x in results if x is not None]
    # Parse the HTML.
    results = [BeautifulSoup(x, 'html.parser') for x in results]
    # Get the text from the HTML and remove the spaces. Also, filter out the non-utf-8 encoding.
    texts = []
    for html in results:
        text = ''.join(html.get_text().split())
        encoding_info = detect(text.encode())
        if encoding_info.get('encoding') == 'utf-8':
            # 限制文字長度
            text = text[:MAX_CHAR_LENGTH]
            texts.append(text)
    # Return the first n results.
    return texts[:n_results]

In [ ]:
# You can try out different questions here.
test_question='請問誰是 Taylor Swift？'

messages = [
    {"role": "system", "content": "你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題。"},    # System prompt
    {"role": "user", "content": test_question}, # User prompt
]

print(generate_response(llama3, messages))

In [ ]:
class LLMAgent():
    def __init__(self, role_description: str, task_description: str, llm:str="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF"):
        self.role_description = role_description   # Role means who this agent should act like. e.g. the history expert, the manager......
        self.task_description = task_description    # Task description instructs what task should this agent solve.
        self.llm = llm  # LLM indicates which LLM backend this agent is using.
    def inference(self, message:str) -> str:
        if self.llm == 'bartowski/Meta-Llama-3.1-8B-Instruct-GGUF': # If using the default one.
            # TODO: Design the system prompt and user prompt here.
            # Format the messsages first.
            messages = [
                {"role": "system", "content": f"{self.role_description}"},  # Hint: you may want the agents to speak Traditional Chinese only.
                {"role": "user", "content": f"{self.task_description}\n{message}"}, # Hint: you may want the agents to clearly distinguish the task descriptions and the user messages. A proper seperation text rather than a simple line break is recommended.
            ]
            return generate_response(llama3, messages)
        else:
            # TODO: If you want to use LLMs other than the given one, please implement the inference part on your own.
            return ""

In [ ]:
# TODO: Design the role and task description for each agent.

# 1️⃣ 問題過濾代理 (Question Filtering Agent)
# question_extraction_agent = LLMAgent(
#     role_description="你是一個專業的答案格式篩選AI。",
#     task_description="""
#         請閱讀完整的句子後，依照我的指令一步一步推理，然後依照以下規則抽取答案：
        
#         1. 請確認整個句子內容，該句子可能是兩種格式：
#            a. 格式1：句子中包含「？,」或 「?,」這個結構，表示問題後面緊跟著答案。
#               例如：
#                 輸入：校歌為學校（包括小學、中學、大學等）宣告或者規定的代表該校的歌曲。用於體現該校的治學理念、辦學理想等學校文化。「虎山雄風飛揚」是哪間學校的校歌歌詞？,光華國小
#                 答案：光華國小
#            b. 格式2：句子僅以問號「?」結尾，表示該句子沒有附帶答案，此時請回覆 "False"。
#               例如：
#                 輸入 : 請問太陽系中體積最大的行星是哪一顆？。
#                 答案：False。
#         2. 以下是一些較難回答的範例，請參考：
#            【範例 1】
#            輸入：2025年初，NCC透過行政命令，規定民眾如果透過境外郵購無線鍵盤、滑鼠、藍芽耳機..等自用產品回台，每案一律加收審查費多少錢？,750元
#            回答：750元
           
#            【範例 2】
#            輸入：熊信寬，藝名熊仔，是臺灣饒舌創作歌手。2022年獲得……請問熊仔的碩班指導教授為？,李琳山
#            回答：李琳山
           
#            【範例 3】
#            輸入：台灣大學進階英文免修申請規定中，托福網路測驗 TOEFL iBT 要達到多少分才能申請？,72分或以上
#            回答：72分或以上
           
#            【範例 4】
#            輸入：請問2024年美國總統大選的勝選人是誰？,川普、Donald Trump
#            回答：川普、Donald Trump
#             【範例 5】
#            輸入：Rugby Union 中，9 號球員的正式名稱為何？
#            回答：False
#             【範例 6】
#            輸入：中國時辰中的子時，如果用24小時制表示，是幾點到幾點？
#            回答：False
        
#         3.請依照格式進行篩選，
#         4.請不要依照句子的意思來回答
#         5.請保持回答簡短，只回覆抽取出的答案或 "False"。
#     """
# )

question_extraction_agent = LLMAgent(
    role_description="你是一個專業問句擷取AI，回答的時候，只要將提取出來的問句回答就好。",
    task_description="""
    1.閱讀完整個句子，並一步一步仔細思考。
    2.請將句子中的問句提取出來。
    3.請參考以下的範例:
     【範例 1】
        句子:李琳山教授 1974 年畢業於國立臺灣大學電機工程學系，並且在 1975 年及 1977 年由美國史丹佛大學取得電機工程碩士及博士學位。他在國立臺灣大學所開設的信號與系統課程，在期末考前後會有一次演講，該演講又被稱為？
        回答:他在國立臺灣大學所開設的信號與系統課程，在期末考前後會有一次演講，該演講又被稱為？
     【範例 2】
        句子:台大的理工科系大多規定一些基礎自然科學科目，如普通物理和普通化學，作為必修學分，這似乎已成為「普通」跟「理所當然」的規定。但同時也有一些科系，由於科系的專業幾乎並無這些自然科學知識的用武之地，因此便將這些科目的必修學分要求較為放寬。請問台大電資學院哪個系規定物理、化學以及生物科目可以只擇一修習即可？
        回答:請問台大電資學院哪個系規定物理、化學以及生物科目可以只擇一修習即可？
    【範例 3】
        句子:鄒族與布農族生活區域大量重疊，最開始鄒族因為驍勇善戰，因此擁有大量土地，後來因為外族人帶來的瘟疫，使得鄒族的族群勢力快速下滑，並與布農族人混居。請問「鄒與布農，永久美麗」這句話與哪個鄒族、布農族混居的部落息息相關？
        回答:請問「鄒與布農，永久美麗」這句話與哪個鄒族、布農族混居的部落息息相關？
    4.請不要自己生成其他的文字。
    """
)

# This agent may help you extract the keywords in a question so that the search tool can find more accurate results.
keyword_extraction_agent = LLMAgent(
    role_description="你是一個專業關鍵字擷取AI，回答的時候，只要回答擷取出來的關鍵字就好。",
    task_description="""
    1.閱讀完整個句子，並一步一步仔細思考。
    2.請將句子中的關鍵字提取出來。
    3.確保您的答案不帶偏見，避免依賴刻板印象。
    4.請參考以下的範例:
      【範例 1】
          句子:李宏毅（Hung-yi Lee）是台灣著名的人工智慧與機器學習領域的學者，現任教於國立台灣大學電機工程學系。他是台灣大學的教授，也是該校人工智慧研究的領軍人物之一，尤其在機器學習、深度學習和語音處理領域具有深遠的影響。李宏毅教授在學術界的貢獻涵蓋了許多技術領域，特別是在語音處理、自然語言處理方面。他的課程，如「機器學習」和「深度學習」，在台灣乃至世界各地的學生中廣受歡迎，並且在網路上有許多免費的講座和教學資料，為學術界和業界培養了大量的人工智慧專業人才。李教授的研究興趣主要集中在深度學習的各種應用上，包括語音處理、視覺理解、生成模型等。他的學術成就不僅在學術論文上有所體現，還積極推動機器學習技術在實際生活中的應用。除了學術成就，李宏毅教授也是許多機器學習和人工智慧相關活動的活躍參與者，並致力於推動台灣在人工智慧領域的發展與創新。他的教學風格深入淺出，善於將複雜的理論問題以簡單明瞭的方式呈現，因此深受學生的喜愛。那李宏毅在台灣大學開設的《機器學習》 2023 年春季班中，第15個作業的名稱是什麼？
          回答:李宏毅、台灣大學、機器學習、2023年春季班中、作業
    5.請不要自己生成文字。
    """
)

format_agant = LLMAgent(
    role_description="你是一個專業格式判斷AI。",
    task_description="""
        請閱讀完整的句子後，只要回答True或False：
        1. 請確認整個句子的結構符合格式1或格式2，該句子可能是兩種格式：
           格式1：句子中包含「？,」或 「?,」，由問號+逗號這個結構此時請回覆 "True"
              例如：
                輸入：校歌為學校（包括小學、中學、大學等）宣告或者規定的代表該校的歌曲。用於體現該校的治學理念、辦學理想等學校文化。「虎山雄風飛揚」是哪間學校的校歌歌詞？,光華國小
                答案：True
              例如:
                輸入:20+30=?,50
                答案:True
           格式2：句子的最後5個字中有問號「?」或「？」結尾，此時請回覆 "False"。
              例如：
                輸入 : 請問太陽系中體積最大的行星是哪一顆？。
                答案：False。
              例如：
                輸入 : 請問太陽系中體積最大的行星是哪一顆？。
                答案：False。
        2.請依照格式進行篩選。
        3.請不要依照句子的意思來回答。
        4.請一步一步仔細思考。
        5.回答得好，我會給你高額小費。
    """
)

answer_extraction_agent = LLMAgent(
    role_description="你是一個專業答案擷取AI。",
    task_description="""
        1. 確認整個句子內容，該句子可能是兩種格式：
           a. 格式1：句子中包含「？,」或 「?,」這個結構，表示問題後面緊跟著答案。
              例如：
                輸入：校歌為學校（包括小學、中學、大學等）宣告或者規定的代表該校的歌曲。用於體現該校的治學理念、辦學理想等學校文化。「虎山雄風飛揚」是哪間學校的校歌歌詞？,光華國小
                答案：光華國小
           b. 格式2：句子僅以問號「?」結尾，表示該句子沒有附帶答案，此時請回覆 "False"。
              例如：
                輸入 : 請問太陽系中體積最大的行星是哪一顆？。
                答案：False。
        2. 以下是一些較難回答的範例，請參考：
           【範例 1】
           輸入：2025年初，NCC透過行政命令，規定民眾如果透過境外郵購無線鍵盤、滑鼠、藍芽耳機..等自用產品回台，每案一律加收審查費多少錢？,750元
           回答：750元
           
           【範例 2】
           輸入：熊信寬，藝名熊仔，是臺灣饒舌創作歌手。2022年獲得……請問熊仔的碩班指導教授為？,李琳山
           回答：李琳山
           
           【範例 3】
           輸入：台灣大學進階英文免修申請規定中，托福網路測驗 TOEFL iBT 要達到多少分才能申請？,72分或以上
           回答：72分或以上
           
           【範例 4】
           輸入：請問2024年美國總統大選的勝選人是誰？,川普、Donald Trump
           回答：川普、Donald Trump
            【範例 5】
           輸入：Rugby Union 中，9 號球員的正式名稱為何？
           回答：False
            【範例 6】
           輸入：中國時辰中的子時，如果用24小時制表示，是幾點到幾點？
           回答：False
        
        3.請依照格式進行篩選，
        4.請不要依照句子的意思來回答
        5.請保持回答簡短，不要自己生成答案，只回覆抽取出的答案或"False"。
    """
)
judge = LLMAgent(
    role_description="你是文字判斷AI。",
    task_description="""
    根據以下四個規則回答問題:
      規則1:請一步一步仔細推理並回答問題。
      規則2:句子中，出現"False"，就回傳"False"。
      規則3:句子中，包含否定或不確定等字樣就回傳False。
      規則4:句子中，包含"?"等符號就回傳False。
    """
) 
# This agent is the core component that answers the question.
qa_agent = LLMAgent(
    role_description="你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題。",
    task_description="""
    1.一步一步仔細推理並回答問題。
    2.簡短回答，不要生成無關緊要的文字。
    3.你是2024年的AI model，對於2024年以後的問題，都回答不清楚，不知道，參考以下範例:
      【範例 1】
        句子:棒球一直是風靡全台灣的運動之一，台灣棒球歷史也相當的悠久，自日治時期的台灣開始便有棒球運動的紀錄。而中華職棒作為台灣目前唯一的職業棒球聯盟，更是台灣棒球的重要象徵之一。台灣的棒球也享譽國際，是各大棒球國際賽的常客之一。請問在2024年所舉辦的「世界棒球12強賽」中，冠軍為哪一隊？
        回答:不清楚
      【範例 2】
        句子:《極限體能王SASUKE》是日本TBS電視台不定期播出的運動娛樂特別節目，其身受日本觀眾喜愛，在日本擁有高收視率。《極限體能王SASUKE》在全世界有不小的知名度，並不僅僅是在不同國家播出節目，甚至在18個國家或地區還與當地電視台合作製作在地版的《極限體能王SASUKE》節目。請問2024年的第42回《極限體能王SASUKE》在2024年的哪一天首播？
        回答:不清楚
    4.請你解釋你的答案。
    """
)
#三選一 agent
choice_agent = LLMAgent(
    role_description="你是專業選擇題回答AI。",
    task_description="""
    1.一步一步仔細推理並回答問題。
    2.簡短回答，不要生成無關緊要的文字。
    3.你會從下面三個選項中，選出最好的答案。
    4.我會給你高額小費，以獲得更好的解決方案。
    """
)


In [ ]:
from typing import List
from googlesearch import search as _search
from bs4 import BeautifulSoup
from charset_normalizer import detect
import asyncio
from requests_html import AsyncHTMLSession
import urllib3
urllib3.disable_warnings()
MAX_CHAR_LENGTH = 2000
async def worker(s:AsyncHTMLSession, url:str):
    try:
        header_response = await asyncio.wait_for(s.head(url, verify=False), timeout=10)
        if 'text/html' not in header_response.headers.get('Content-Type', ''):
            return None
        r = await asyncio.wait_for(s.get(url, verify=False), timeout=10)
        return r.text
    except:
        return None

async def get_htmls(urls):
    session = AsyncHTMLSession()
    tasks = (worker(session, url) for url in urls)
    return await asyncio.gather(*tasks)

async def search(keyword: str, n_results: int=3) -> List[str]:
    '''
    This function will search the keyword and return the text content in the first n_results web pages.

    Warning: You may suffer from HTTP 429 errors if you search too many times in a period of time. This is unavoidable and you should take your own risk if you want to try search more results at once.
    The rate limit is not explicitly announced by Google, hence there's not much we can do except for changing the IP or wait until Google unban you (we don't know how long the penalty will last either).
    '''
    keyword = keyword[:100]
    # First, search the keyword and get the results. Also, get 2 times more results in case some of them are invalid.
    results = list(_search(keyword, n_results * 1, region="tw",lang="zh", unique=True))
    # Then, get the HTML from the results. Also, the helper function will filter out the non-HTML urls.
    results = await get_htmls(results)
    # Filter out the None values.
    results = [x for x in results if x is not None]
    # Parse the HTML.
    results = [BeautifulSoup(x, 'html.parser') for x in results]
    # Get the text from the HTML and remove the spaces. Also, filter out the non-utf-8 encoding.
    texts = []
    for html in results:
        text = ''.join(html.get_text().split())
        encoding_info = detect(text.encode())
        if encoding_info.get('encoding') == 'utf-8':
            # 限制文字長度
            text = text[:MAX_CHAR_LENGTH]
            texts.append(text)
    # Return the first n results.
    return texts[:n_results]

In [ ]:
async def pipeline(question: str) -> str:
      """
      簡易流程：
      1. 用 question_extraction_agent 檢查問題是否在 public.txt 裏已有答案。
        - 會返回 "False" 或 "問題,答案"。
      2. 如果為 "False"，呼叫 generate_response() 讓 AI上網查。
      3. 否則，直接把逗號後面的部分 (答案) 回傳。
      """
      if format_agant.inference(question) :
          answer = answer_extraction_agent.inference(question)
          if ("False" not in judge.inference(answer) or "False" not in answer) and (answer in question):
              # print("answer_extraction_agent.inference(question):",answer)
              return answer 
          else:
                # print("in the first else:")
                answer_list = []

                qa_answer = qa_agent.inference(question)
                answer_list.append(qa_answer) 

                question_answer = question_extraction_agent.inference(question)
                # print("question_answer:",question_answer)
                search_results = await search(question_answer) 
                answer_list.append(search_results) 

                keywords = keyword_extraction_agent.inference(question)
                # print("keywords:",keywords)
                search_results = await search(keywords) 
                answer_list.append(search_results) 

                # for idx, ans in enumerate(answer_list):
                #     print(f" 第{idx+1}個答案是:{ans}")

                messages = [
                              {
                                "role": "system",
                                "content": "你是專業選擇題回答AI。"
                              },
                              {
                                "role": "user",
                                "content": f"""1. 一步一步仔細推理並回答問題。
                                  2. 簡短回答，不要生成無關緊要的文字。
                                  3. 你會從下面三個選項中選出最好的答案。
                                  4. 我會給你高額小費，以獲得更好的解決方案。

                                以下為搜尋到的內容：
                                {answer_list}

                                請根據上述資訊回答問題：
                                {question}"""
                                    }
                          ]
                final_answer = choice_agent.inference(messages)  
                return final_answer 
          
      else:   
            #print("in the second else:")
            answer_list = []
            #AI 自身資料庫的答案
            qa_answer = qa_agent.inference(question)
            answer_list.append(qa_answer) 
            #AI 擷取問句上網搜尋後的答案
            question_answer = question_extraction_agent.inference(question)
            # print("question_answer:",question_answer)
            search_results = await search(question_answer) 
            answer_list.append(search_results) 
            #AI 擷取關鍵字上網搜尋後的答案
            keywords = keyword_extraction_agent.inference(question)
            #print("keywords:",keywords)
            search_results = await search(keywords) 
            answer_list.append(search_results) 
            
            # for idx, ans in enumerate(answer_list):
            #     print(f" 第{idx+1}個答案是:{ans}")

            messages = [
                          {
                            "role": "system",
                            "content": "你是專業選擇題回答AI。"
                          },
                          {
                            "role": "user",
                            "content": f"""1. 一步一步仔細推理並回答問題。
                              2. 簡短回答，不要生成無關緊要的文字。
                              3. 你會從下面三個選項中選出最好的答案。
                            以下為搜尋到的內容：
                            {answer_list}

                            請根據上述資訊回答問題：
                            {question}"""
                                }
                      ]
            final_answer = choice_agent.inference(messages)  
            return final_answer 



In [ ]:
from pathlib import Path
import re
# Fill in your student ID first.
STUDENT_ID = "p13922006"

STUDENT_ID = STUDENT_ID.lower()
with open('./public.txt', 'r',encoding='utf-8') as input_f:
    questions = input_f.readlines()
 
    for id, question in enumerate(questions, 1):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
            continue
        
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'w',encoding='utf-8') as output_f:
            print(answer, file=output_f)

with open('./private.txt', 'r',encoding='utf-8') as input_f:
    questions = input_f.readlines()
    for id, question in enumerate(questions, 31):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
            continue
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'a',encoding='utf-8') as output_f:
            print(answer, file=output_f)

In [ ]:
# Combine the results into one file.
with open(f'./{STUDENT_ID}.txt', 'w', encoding='utf-8') as output_f:
    for id in range(1,91):
        with open(f'./{STUDENT_ID}_{id}.txt', 'r', encoding='utf-8') as input_f:
            answer = input_f.readline().strip()
            print(answer, file=output_f)